# ScyllaDB Store - CRUD Operations Testing

This notebook demonstrates all CRUD operations with the ScyllaDB store implementation.

## Setup

First, let's import the necessary modules and create a store instance.

In [ ]:
import asyncio
from datetime import datetime
from scylladb_store import AsyncScyllaDBStore, TTLConfig, GetOp, PutOp, SearchOp, ListNamespacesOp

# Configuration
CONTACT_POINTS = ["127.0.0.1"]
KEYSPACE = "test_crud_store"

print("✓ Imports successful")

In [ ]:
# Create store instance
store = None

async def init_store():
    global store
    from cassandra.cluster import Cluster
    
    # Create cluster and session
    cluster = Cluster(CONTACT_POINTS)
    session = await asyncio.get_event_loop().run_in_executor(None, cluster.connect)
    
    # Create store
    store = AsyncScyllaDBStore(
        session=session,
        keyspace=KEYSPACE,
        ttl=TTLConfig(refresh_on_read=True)
    )
    
    # Setup database
    await store.setup()
    print(f"✓ Store initialized with keyspace: {KEYSPACE}")

await init_store()

## CREATE Operations

Test creating/storing items in the database.

In [ ]:
# Create a single item
await store.aput(
    namespace=("users", "123"),
    key="profile",
    value={"name": "Alice", "email": "alice@example.com", "age": 30}
)
print("✓ Created user profile for Alice")

In [ ]:
# Create multiple items
users = [
    {"namespace": ("users", "456"), "key": "profile", "value": {"name": "Bob", "email": "bob@example.com", "age": 25}},
    {"namespace": ("users", "789"), "key": "profile", "value": {"name": "Charlie", "email": "charlie@example.com", "age": 35}},
]

for user in users:
    await store.aput(**user)
    print(f"✓ Created user profile for {user['value']['name']}")

In [ ]:
# Create items in different namespace
projects = [
    {"namespace": ("projects", "p1"), "key": "metadata", "value": {"title": "Project Alpha", "status": "active", "score": 4.5}},
    {"namespace": ("projects", "p2"), "key": "metadata", "value": {"title": "Project Beta", "status": "completed", "score": 5.0}},
    {"namespace": ("projects", "p3"), "key": "metadata", "value": {"title": "Project Gamma", "status": "active", "score": 3.8}},
]

for project in projects:
    await store.aput(**project)
    print(f"✓ Created project: {project['value']['title']}")

In [ ]:
# Create item with TTL (expires in 5 minutes)
await store.aput(
    namespace=("temp", "session-123"),
    key="data",
    value={"token": "abc123", "expires": "in 5 minutes"},
    ttl=5.0
)
print("✓ Created temporary session with 5-minute TTL")

## READ Operations

Test reading/retrieving items from the database.

In [ ]:
# Read a single item
item = await store.aget(("users", "123"), "profile")
print(f"Retrieved item:")
print(f"  Name: {item.value['name']}")
print(f"  Email: {item.value['email']}")
print(f"  Age: {item.value['age']}")
print(f"  Created at: {item.created_at}")
print(f"  Updated at: {item.updated_at}")

In [ ]:
# Read non-existent item
missing = await store.aget(("users", "999"), "profile")
print(f"Non-existent item: {missing}")

In [ ]:
# Search all users
users = await store.asearch(("users",), limit=10)
print(f"\nFound {len(users)} users:")
for user in users:
    print(f"  - {user.value['name']} (age: {user.value['age']})")

In [ ]:
# Search with filter (age > 30)
results = await store.asearch(
    ("users",),
    filter={"age": {"$gt": 30}},
    limit=10
)
print(f"\nUsers with age > 30:")
for user in results:
    print(f"  - {user.value['name']} (age: {user.value['age']})")

In [ ]:
# Search active projects
active_projects = await store.asearch(
    ("projects",),
    filter={"status": "active"},
    limit=10
)
print(f"\nActive projects:")
for project in active_projects:
    print(f"  - {project.value['title']} (score: {project.value['score']})")

In [ ]:
# List all namespaces
namespaces = await store.alist_namespaces()
print(f"\nAll namespaces ({len(namespaces)}):")
for ns in namespaces:
    print(f"  - {ns}")

In [ ]:
# List namespaces with prefix
user_namespaces = await store.alist_namespaces(prefix=("users",))
print(f"\nUser namespaces:")
for ns in user_namespaces:
    print(f"  - {ns}")

In [ ]:
# List namespaces with max_depth
truncated = await store.alist_namespaces(max_depth=1)
print(f"\nNamespaces (max depth 1):")
for ns in truncated:
    print(f"  - {ns}")

## UPDATE Operations

Test updating existing items.

In [ ]:
# Update a single field (actually replaces entire value)
await store.aput(
    namespace=("users", "123"),
    key="profile",
    value={"name": "Alice", "email": "alice@example.com", "age": 31}  # Age updated
)
print("✓ Updated Alice's age to 31")

# Verify update
item = await store.aget(("users", "123"), "profile")
print(f"  New age: {item.value['age']}")

In [ ]:
# Update project status
await store.aput(
    namespace=("projects", "p1"),
    key="metadata",
    value={"title": "Project Alpha", "status": "completed", "score": 4.5}
)
print("✓ Updated Project Alpha status to completed")

# Verify
item = await store.aget(("projects", "p1"), "metadata")
print(f"  New status: {item.value['status']}")

In [ ]:
# Update multiple items
updates = [
    {"namespace": ("users", "456"), "key": "profile", "value": {"name": "Bob", "email": "bob.smith@example.com", "age": 26}},
    {"namespace": ("users", "789"), "key": "profile", "value": {"name": "Charlie", "email": "charlie.brown@example.com", "age": 36}},
]

for update in updates:
    await store.aput(**update)
    print(f"✓ Updated {update['value']['name']}'s profile")

## DELETE Operations

Test deleting items from the database.

In [ ]:
# Delete a single item
await store.adelete(("users", "789"), "profile")
print("✓ Deleted Charlie's profile")

# Verify deletion
deleted = await store.aget(("users", "789"), "profile")
print(f"  Item exists after deletion: {deleted is not None}")

In [ ]:
# Verify remaining users
users = await store.asearch(("users",), limit=10)
print(f"\nRemaining users ({len(users)}):")
for user in users:
    print(f"  - {user.value['name']}")

In [ ]:
# Delete multiple items
to_delete = [
    (("projects", "p2"), "metadata"),
    (("temp", "session-123"), "data"),
]

for namespace, key in to_delete:
    await store.adelete(namespace, key)
    print(f"✓ Deleted {namespace} / {key}")

In [ ]:
# Verify remaining projects
projects = await store.asearch(("projects",), limit=10)
print(f"\nRemaining projects ({len(projects)}):")
for project in projects:
    print(f"  - {project.value['title']}")

## BATCH Operations

Test executing multiple operations in a single batch.

In [ ]:
# Batch: Mix of GET, PUT, SEARCH operations
ops = [
    GetOp(namespace=("users", "123"), key="profile"),
    GetOp(namespace=("users", "456"), key="profile"),
    PutOp(
        namespace=("users", "999"),
        key="profile",
        value={"name": "David", "email": "david@example.com", "age": 28}
    ),
    SearchOp(
        namespace_prefix=("users",),
        limit=10
    ),
    ListNamespacesOp(
        match_conditions=(("users",), None, None),
        limit=10
    )
]

results = await store.abatch(ops)
print(f"✓ Executed {len(ops)} operations in batch\n")

print("Results:")
print(f"  1. GET user 123: {results[0].value['name'] if results[0] else 'Not found'}")
print(f"  2. GET user 456: {results[1].value['name'] if results[1] else 'Not found'}")
print(f"  3. PUT user 999: Success")
print(f"  4. SEARCH users: Found {len(results[3])} users")
print(f"  5. LIST namespaces: Found {len(results[4])} user namespaces")

In [ ]:
# Verify the batch PUT worked
item = await store.aget(("users", "999"), "profile")
print(f"\nNew user from batch:")
print(f"  Name: {item.value['name']}")
print(f"  Email: {item.value['email']}")
print(f"  Age: {item.value['age']}")

## Advanced Filtering

Test different filter operators.

In [ ]:
# Equal to
results = await store.asearch(
    ("users",),
    filter={"name": "Alice"},
    limit=10
)
print(f"Users named Alice: {len(results)}")

In [ ]:
# Greater than or equal
results = await store.asearch(
    ("users",),
    filter={"age": {"$gte": 28}},
    limit=10
)
print(f"\nUsers with age >= 28:")
for user in results:
    print(f"  - {user.value['name']} (age: {user.value['age']})")

In [ ]:
# Less than
results = await store.asearch(
    ("users",),
    filter={"age": {"$lt": 30}},
    limit=10
)
print(f"\nUsers with age < 30:")
for user in results:
    print(f"  - {user.value['name']} (age: {user.value['age']})")

In [ ]:
# Multiple conditions
results = await store.asearch(
    ("projects",),
    filter={
        "status": "active",
        "score": {"$gte": 4.0}
    },
    limit=10
)
print(f"\nActive projects with score >= 4.0:")
for project in results:
    print(f"  - {project.value['title']} (score: {project.value['score']})")

## Pagination

Test pagination with offset and limit.

In [ ]:
# Get first page (2 items)
page1 = await store.asearch(("users",), limit=2, offset=0)
print("Page 1 (limit=2, offset=0):")
for user in page1:
    print(f"  - {user.value['name']}")

In [ ]:
# Get second page
page2 = await store.asearch(("users",), limit=2, offset=1)
print("\nPage 2 (limit=2, offset=1):")
for user in page2:
    print(f"  - {user.value['name']}")

In [ ]:
# Get second page
page2 = await store.asearch(("users",), limit=2, offset=2)
print("\nPage 2 (limit=2, offset=2):")
for user in page2:
    print(f"  - {user.value['name']}")

## Summary

Display final state of the database.

In [ ]:
# Get all users
users = await store.asearch(("users",), limit=100)
print(f"Total users: {len(users)}")
for user in users:
    print(f"  - {user.value['name']} (age: {user.value['age']}, email: {user.value['email']})")

In [ ]:
# Get all projects
projects = await store.asearch(("projects",), limit=100)
print(f"\nTotal projects: {len(projects)}")
for project in projects:
    print(f"  - {project.value['title']} (status: {project.value['status']}, score: {project.value['score']})")

In [ ]:
# All namespaces
namespaces = await store.alist_namespaces()
print(f"\nTotal namespaces: {len(namespaces)}")
for ns in namespaces:
    print(f"  - {ns}")

## Cleanup

Optionally clean up all test data.

In [ ]:
# Uncomment to clean up all data
async def cleanup():
    # Get all namespaces
    namespaces = await store.alist_namespaces()
    
    # Delete all items in each namespace
    for ns in namespaces:
        items = await store.asearch(ns, limit=1000)
        for item in items:
            await store.adelete(item.namespace, item.key)
            print(f"Deleted: {item.namespace} / {item.key}")
    
    print("\n✓ All data cleaned up")

await cleanup()